In [2]:
import numpy as np
import xarray as xr
import os
import ot

In [5]:
root = "../../real_data/02_GrotteDeLaCascadeDeMotiers/process/"
fp_list = os.listdir(os.path.normpath(os.path.join(root, "sections_as_polylines")))

In [6]:
fp_sections = [elem for elem in fp_list if ("ThirdGallery_1_reordered_2d" in elem) and ("ply" not in elem)]

In [7]:
sections = [np.loadtxt(os.path.normpath(os.path.join(root,"sections_as_polylines", fp))) for fp in fp_sections]

plane_vectors = np.loadtxt(os.path.normpath(os.path.join(root,"planes_as_vectors","ThirdGallery_1_Planes.txt")))
plane_origins = np.loadtxt(os.path.normpath(os.path.join(root,"planes_as_vectors","ThirdGallery_1_Planes_Origin.txt")))
curvilinear_pos = np.round(np.cumsum(np.linalg.norm(np.diff(plane_origins, axis = 0), axis = 1)), 2)
curvilinear_pos = np.pad(curvilinear_pos, (1, 0), "constant", constant_values = (0, 0))

In [10]:
datasets = []

for i in range(len(sections)):
    ds = xr.DataArray(sections[i], dims=("point", "coords_xz"))
    ds.attrs["position"] = plane_origins[i]
    ds.attrs["normal"] = plane_vectors[i]
    ds.attrs["curvilinear"] = curvilinear_pos[i]
    ds.to_netcdf(os.path.normpath(os.path.join(root, "sections_as_polylines", f"ThirdGallery_1_{i:03d}.nc")))
    datasets.append(ds)

In [26]:
reg = 1e-2
X1_idx = np.random.choice(sections[0].astype(np.float32), 200, replace = False)
X2 = np.random.choice(sections[1].astype(np.float32), 200, replace = False)
a1 = ot.unif(len(X1)).astype(np.float32)
a2 = ot.unif(len(X2)).astype(np.float32)

n_samples = len(X1)

XB_init = np.random.randn(n_samples, 2).astype(np.float32)

XB = ot.bregman.free_support_sinkhorn_barycenter(measures_locations=[X1, X2], 
                                                 measures_weights=[a1, a2], 
                                                 weights=np.array([0.5, 0.5]).astype(np.float32),X_init=XB_init, reg = reg)


ValueError: a must be 1-dimensional